# Algoritimo Simplex utilizando Python and Numpy. #

Simplex é um método Iterativo para solução de problemas de Otimização Linear, com um número finito de passos leva à solução ótima do problema.

## Algoritmo do Método Simplex ##

### Para a resolução de um problema usando simplex são utilizados os seguintes passos: ###
1. Adicionar variáveis de folga para cada uma das desigualdades;
2. Montar uma tabela para os cálculos, colocando todos os coeficientes das restrições e da função objetivo modificada;
3. Determinar uma solução básica factível inicial e/ou verificar se a solução é ótima, se todos os valores da linha correspondente à função objetivo são positivos ou nulos, então a solução atual é ótima;
4. Verificar qual variável entra na base, uma candidata a entrar na base é a variável que apresenta o valor mais negativo na linha da função objetivo. Isto é, apresenta maior contribuição para o aumento da do valor da função objetivo. Esta será a coluna pivô;
5. Determinar o elemento que sai da base, dividir os
valores da coluna b, pelos valores das respectivas
variáveis na coluna pivô. A variável referente ao menor
valor da divisão será a que sairá da base.
6. Recalcular a tabela fazendo com que a coluna
referente à variável que entra na base se torne uma coluna
identidade, idêntica à coluna da variável que sai da base.
7. Retornar ao Passo **3**.

## Simplex - AC1 ##

Agora introduzido no assunto é o momento de entender o propózito deste código.
Temos como objetivo escrever um código que calcule o método **Simplex** para problemas de **maximização**, que tem como objetivo a avaliação dos integrantes envolvidos. 

**Linguagem de programação utilizada:** Python

**Entregáveis:**
- código-fonte;
- notebook com código comentado.

**Arquivo de entrada:**

- deve respeitar o formato padrão desse [exemplo de entrada](./exemplo_01_entrada.txt), sabendo que TODAS as restrições possuem sinal de menor ou igual (<=);
- considerar que o arquivo de entrada estará sempre na mesma pasta que o código.

## Código explicado ##

1. Será necessário um metodo de leitura de arquivo para a entrada dos dados:
> Considerando que o arquivo que será lido esá na pasta raiz e que existe apenas um arquivo `.txt` 

In [1]:
import os

def leitura_arquivo():
    linhas = []
    arquivos = os.listdir('.')
    for f in arquivos:
        if f.endswith(".txt"):
            f = open(f)
            for linha in f:
                linhas.append(linha.replace("\n", "").split(" "))
    variaveis = len(linhas[0])
    restricoes = len(linhas) - 1
    return linhas, variaveis, restricoes

2. Com os valores obtidos com o arquivo será gerado uma matriz numpy com linhas suficientes para cada restrição mais a função objetivo e colunas suficientes para as variáveis, variáveis de folga, M (máx.) E o valor correspondente.

In [2]:
import numpy as np

def geracao_tabela(variaveis, restricoes):
    tabela = np.zeros((restricoes + 1, variaveis + restricoes + 2))
    return tabela

3. Agora que a tabela está montada é hora de preparar a função objetivo e as restrições. 

In [3]:
def obtem_objetivo_restricoes(linhas):
    objetive = []
    restricoes = []
    for linha in linhas:
        if linhas.index(linha) == 0:
            objetive = linha
        else:
            restricoes.append(linha)
    return objetive, restricoes

4. Os valores obtidos no arquivo deveram ser convertidos de string em variáveis ​​float

In [4]:
def converter(equacao):
    equacao = [float(valor) for valor in equacao]
    return equacao

5. Com esses valores do arquivo e com o metodo escrito anteriormente, agora é o momento de preencher as restrições do problema na tabela. 

In [5]:
def preencher_restricoes(tabela,restricoes):
    for restricao in restricoes:
        quant_colunas = len(tabela[0, :])
        quant_linhas = len(tabela[:, 0])
        valores_tabela = quant_colunas - quant_linhas -1
        j = 0
        while j < quant_linhas:
            verifica = tabela[j, :]
            total = 0
            for i in verifica:
                total += float(i ** 2)
            if total == 0:
                linha = verifica
                break
            j += 1
        restricao = converter(restricao)
        i = 0
        while i < len(restricao) - 1:
            linha[i] = restricao[i]
            i += 1
        linha[-1] = restricao[-1]
        linha[valores_tabela + j] = 1

6. Com a tabela já preenchida com as restrições é hora de adicionar a função objetivo à tabela.

In [6]:
def preencher_objetivo(tabela,equacao):
    equacao = converter(equacao)
    quant_linhas = len(tabela[:, 0])
    linha = tabela[quant_linhas - 1, :]
    for i in range(len(equacao)):
        linha[i] = equacao[i] * -1
    linha[-2] = 1

7. Seguindo os passos para a resolução de um problema utilizando simplex, com os passos `1` e `2` já finalizados, seguindo então para o paso `3` para verificar se todos os valores da linha correspondente à função objetivo são positivos ou nulos.

In [7]:
def valida_func_linha_objetivo(tabela):
    quant_linhas = len(tabela[:, 0])
    m = min(tabela[quant_linhas - 1, :-1])
    if m >= 0:
        return False
    else:
        return True

8. Seguindo então com a resolução, agora é hora de verificar qual variável entra na base, uma candidata a entrar na base é a variável que apresenta o valor mais negativo na linha da função objetivo. Isto é, apresenta maior contribuição para o aumento do valor da função objetivo. Esta será a coluna pivô.

8.1. Primeiramente deve-se validar de ainda existem valores nulos ou negativos.

In [8]:
def valida_func_coluna_objetivo(tabela):
    m = min(tabela[:-1, -1])
    if m >= 0:
        return False
    else:
        return True

8.2. Feito isso é necessário localizar o elemento negativo na linha da função objetivo.

In [9]:
def elemento_negativo_func_coluna_objetivo(tabela):
    quant_linhas = len(tabela[:, 0])
    m = min(tabela[quant_linhas - 1, :-1])
    if m <= 0:
        n = np.where(tabela[quant_linhas - 1, :-1] == m)[0][0]
    else:
        n = None
    return n

9. Com isso é o momento de determinar o elemento que sai da base, dividir os valores da coluna b, pelos valores das respectivas variáveis na coluna pivô. A variável referente ao menor valor da divisão será a que sairá da base.

In [10]:
def localizar_pivo(tabela):
    if valida_func_linha_objetivo(tabela):
        total = []
        n = elemento_negativo_func_coluna_objetivo(tabela)
        for i, b in zip(tabela[:-1, n], tabela[:-1, -1]):
            if b/i > 0 and i ** 2 > 0:
                total.append(b / i)
            else:
                total.append(None)
        m = min(filter(lambda x: x is not None, total))
        index = total.index(m)
        return [index, n]

10. Recalcular a tabela fazendo com que a coluna referente à variável que entra na base se torne uma coluna identidade, idêntica à coluna da variável que sai da base. O que será preciso fazer no final e retornar a tabela atualizada.

In [11]:
def pivo(linha, coluna, tabela):
    quant_linhas = len(tabela[:, 0])
    quant_colunas = len(tabela[0, :])
    tabela_imagem = np.zeros((quant_linhas, quant_colunas))
    linha_transformacao = tabela[linha, :]
    if tabela[linha, coluna] ** 2 > 0:
        valor = 1 / tabela[linha, coluna]
        linha_transformada = linha_transformacao * valor
        for i in range(len(tabela[:, coluna])):
            k = tabela[i, :]
            c = tabela[i, coluna]
            if list(k) == list(linha_transformacao):
                continue
            else:
                tabela_imagem[i, :] = list(k - linha_transformada * c)
        tabela_imagem[linha, :] = list(linha_transformada)
    return tabela_imagem

11. Esta será uma função que irá gerar o número necessário de variáveis ​​x1, x2,… xn. 

In [12]:
def gerar_variaveis(tabela):
    quant_colunas = len(tabela[0, :])
    quant_linhas = len(tabela[:, 0])
    variaveis = quant_colunas - quant_linhas -1
    lista_variaveis = []
    for i in range(variaveis):
        lista_variaveis.append('x' + str(i +1))
    return lista_variaveis

Agora, agradeça a si mesmo por sua incrível paciência - finalmente é hora de colocar todos os blocos de construção juntos e criar as funções de maximização e minimização! Essas funções parecerão muito semelhantes, ambas usarão loops while para determinar se 1+ pivô é necessário, localize o elemento pivô, gire sobre ele e continue o processo até que todos os elementos negativos tenham sido removidos da última coluna e linha. Em seguida, as variáveis serão geradas para x1 a xn e valores atribuídos de acordo com suas posições no quadro. Além disso, max receberá seu valor apropriado. Por último, a função retornará o máximo e as variáveis na forma de dicionário.

In [13]:
def maximizacao(linhas, variaveis, restricoes):
    tabela = geracao_tabela(variaveis, restricoes)
    objetive, restricoes = obtem_objetivo_restricoes(linhas)
    preencher_restricoes(tabela,restricoes)
    preencher_objetivo(tabela,objetive)
    while valida_func_linha_objetivo(tabela) == True:
        tabela = pivo(localizar_pivo(tabela)[0], localizar_pivo(tabela)[1], tabela)
    quant_colunas = len(tabela[0, :])
    quant_linhas = len(tabela[:, 0])
    quant_var = quant_colunas - quant_linhas -1
    i = 0
    valores = {}
    for i in range(quant_var):
        coluna = tabela[:, i]
        soma = sum(coluna)
        maximo = max(coluna)
        if float(soma) == float(maximo):
            valor_localizado = np.where(coluna == maximo)[0][0]
            valores[gerar_variaveis(tabela)[i]] = tabela[valor_localizado, -1]
        else:
            valores[gerar_variaveis(tabela)[i]] = 0
    valores['maximo'] = tabela[-1, -1]
    return valores

12. Escrita dos resultados no arquivo

In [14]:
def escrita_arquivo(resultado):
    linhas = []
    arquivos = os.listdir(".")
    for f in arquivos:
        if f.endswith(".txt"):
            f = open(f, "a+")
            f.write("\n")
            f.write(str(resultado))
            f.close()

In [15]:
if __name__ == "__main__":
    linhas, variaveis, restricoes = leitura_arquivo()
    valores = maximizacao(linhas, variaveis, restricoes)
    print(valores)
    escrita_arquivo(valores)

{'x1': 2.0, 'x2': 6.0, 'maximo': 36.0}
